# Georgian Spellchecker: Inference Notebook

Demonstration of trained CharSeq2Seq model in action, see `data_and_training.ipynb`.

We will load the final weights and test the model's ability to correct various types of Georgian typos, including deletions, duplications, swaps, and random character substitutions.

## Setup

First, we define the vocabulary and encoding logic used during training to ensure the input tokens match exactly.

In [4]:
GEORGIAN_LETTERS = list("აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ-")
special_tokens = ["<PAD>", "<SOS>", "<EOS>"]
vocab = special_tokens + GEORGIAN_LETTERS

char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = {i: c for i, c in enumerate(vocab)}
vocab_size = len(vocab)

def encode_word(word):
    return [char2idx["<SOS>"]] + [char2idx[c] for c in word if c in char2idx] + [char2idx["<EOS>"]]

## Model: `correct_word()`
I am placing correct word generation and translation inside model because why not.

In [5]:
import torch
from torch import nn

class CharSeq2Seq(nn.Module):
    def __init__(self, embed_dim=64, hidden_dim=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder = nn.LSTM(embed_dim, hidden_dim, num_layers=2, batch_first=True, dropout=0.2)
        self.decoder = nn.LSTM(embed_dim, hidden_dim, num_layers=2, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    @torch.no_grad()
    def correct_word(self, word: str, max_len: int = 50) -> str:
        self.eval()
        device = next(self.parameters()).device
        src = torch.tensor([encode_word(word)], device=device)

        # Encoding
        embedded_src = self.embedding(src)
        _, hidden = self.encoder(embedded_src)

        # Decoding (Greedy)
        decoder_input = torch.tensor([[char2idx["<SOS>"]]], device=device)
        decoded_chars = []

        for _ in range(max_len):
            embedded_input = self.embedding(decoder_input)
            output, hidden = self.decoder(embedded_input, hidden)
            next_token = self.fc(output).argmax(dim=-1)
            token_id = next_token.item()

            if token_id == char2idx["<EOS>"]:
                break
            if token_id != char2idx["<SOS>"] and token_id != char2idx["<PAD>"]:
                decoded_chars.append(idx2char[token_id])

            decoder_input = next_token.view(1, -1)

        return "".join(decoded_chars)

## Implementation of correct_word_wrapper

In [6]:
def correct_word_standalone(word, model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = CharSeq2Seq()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)

    return model.correct_word(word)

## Action

In [8]:
model_path = "./trained/georgian_spellcheck_model_final.pth"

test_words = [
    # --- Correct Words ---
    "გამარჯობა", "რობოტი", "გიორგი", "გზა", "მწვრთნელი", "იასამანი", "ჰიდროელექტროსადგური",
    # --- Deletions ---
    "გამარჯობ", "მზეზ", "სოკ", "გამ", "ფანჯრა",
    # --- Duplications ---
    "მთვარეე", "მზეზზე", "მგელიი", "წიგნნნი", "გოგონნა",
    # --- Swaps & Substitutions ---
    "ცამოდი", "დავთით", "ყტემალი", "მოტოციკკეტი",
    # --- Edge Cases ---
    "სოო", ""
]


print("Spellcheck results:\n")
for w in test_words:
    corrected = correct_word_standalone(w, model_path)
    print(f"-    {w} -> {corrected}")

Spellcheck results:

-    გამარჯობა -> გამარჯობა
-    რობოტი -> რობოტი
-    გიორგი -> გიორგი
-    გზა -> გზა
-    მწვრთნელი -> მწვრთმელი
-    იასამანი -> იასამანი
-    ჰიდროელექტროსადგური -> ჰიდროელექტროსადგური
-    გამარჯობ -> გამარჯობა
-    მზეზ -> მზეზე
-    სოკ -> სოკო
-    გამ -> გამო
-    ფანჯრა -> ფანჯარა
-    მთვარეე -> მთვარე
-    მზეზზე -> მზეზე
-    მგელიი -> მგელი
-    წიგნნნი -> წიგნი
-    გოგონნა -> გოგონა
-    ცამოდი -> ამოდი
-    დავთით -> დათვით
-    ყტემალი -> ტყემალი
-    მოტოციკკეტი -> მოტოციკლეტი
-    სოო -> სოლო
-     -> ალ


While the model is not perfect, it successfully demonstrates how a character-level Seq2Seq architecture can master Georgian orthography. By learning the language's structural patterns, it effectively repairs common human errors.

Even when faced with significant corruption, the model often recovers the intended root, proving it has moved beyond simple memorization toward a functional understanding of Georgian word formation.